<a href="https://colab.research.google.com/github/BONFACEVULU/AI-Coursework-2025/blob/main/Diamonds_Price_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving diamonds.csv to diamonds (1).csv


In [ ]:
import pandas as pd

diamonds_model = pd.read_csv('diamonds.csv')

In [ ]:
#QUESTION 3
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

X = diamonds_model.drop('price', axis=1)
y = diamonds_model['price']


c_cols = ['cut', 'color', 'clarity']
n_cols = ['carat', 'depth', 'table', 'x', 'y', 'z']


preprocessor = ColumnTransformer([
    ('category', OneHotEncoder(drop='first'), c_cols)
], remainder='passthrough')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

model_lr.fit(X_train, y_train)


y_pred = model_lr.predict(X_test)

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Linear Regression R²: {r2:.4f}")
print(f"Linear Regression RMSE: {rmse:.2f}")

Linear Regression R²: 0.9193
Linear Regression RMSE: 1132.36


In [ ]:
#QUESTION 4
from sklearn.decomposition import PCA

# numeric features
X_num = diamonds_model[num_features]
y_pca = diamonds_model['price']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_num)

pca = PCA()
pca.fit(X_scaled)

explained_variance = pd.DataFrame({
    "Feature": num_features,
    "Variance Ratio": pca.explained_variance_ratio_
})
explained_variance

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Regression using PCA features
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y_pca, test_size=0.2, random_state=42)
pca_model = LinearRegression()
pca_model.fit(X_train_pca, y_train_pca)
y_pred_pca = pca_model.predict(X_test_pca)

print("PCA-based Linear Regression R²:", r2_score(y_test_pca, y_pred_pca))
print("RMSE:", np.sqrt(mean_squared_error(y_test_pca, y_pred_pca)))


PCA-based Linear Regression R²: 0.8026594063583394
RMSE: 1771.1827071781886


In [ ]:
# QUESTION 5: Lasso and Ridge Regression Models

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score

diamonds = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv")
diamonds_model = diamonds.sample(n=12500, random_state=42)

X = diamonds_model.drop('price', axis=1)
y = diamonds_model['price']

c_cols = ['cut', 'color', 'clarity']

preprocessor = ColumnTransformer([
    ('category', OneHotEncoder(drop='first'), c_cols)
], remainder='passthrough')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_lasso = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', Lasso(alpha=0.1, random_state=42))
])

model_ridge = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', Ridge(alpha=1.0, random_state=42))
])

model_lasso.fit(X_train, y_train)
y_pred_lasso = model_lasso.predict(X_test)
r2_lasso = r2_score(y_test, y_pred_lasso)
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))

model_ridge.fit(X_train, y_train)
y_pred_ridge = model_ridge.predict(X_test)
r2_ridge = r2_score(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))

print("Lasso Regression R²:", round(r2_lasso, 4))
print("Lasso Regression RMSE:", round(rmse_lasso, 2))
print("Ridge Regression R²:", round(r2_ridge, 4))
print("Ridge Regression RMSE:", round(rmse_ridge, 2))


Lasso Regression R²: 0.9232
Lasso Regression RMSE: 1103.72
Ridge Regression R²: 0.923
Ridge Regression RMSE: 1105.4


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.298e+09, tolerance: 1.577e+07
  model = cd_fast.enet_coordinate_descent(


In [ ]:
# Question 6
print("Model Comparison Summary")
print("-------------------------")
print(f"Linear Regression R²: {r2:.4f}, RMSE: {rmse:.2f}")
print(f"PCA Regression R²: {0.8027:.4f}, RMSE: {1771.18:.2f}")
print(f"Lasso Regression R²: {r2_lasso:.4f}, RMSE: {rmse_lasso:.2f}")
print(f"Ridge Regression R²: {r2_ridge:.4f}, RMSE: {rmse_ridge:.2f}")

print("\nHighest Accuracy: Ridge Regression")
print("Reason: Ridge controls coefficient magnitudes and mitigates overfitting, "
      "leading to slightly higher generalization performance.")

The 4 models developed had the following performance summary:
Model                      R2 Score         RMSE
Linear Regression          0.9193         1132.36
PCA + Linear Regression    0.9193         1132.36
Lasso Regression           0.9193         1132.36
Ridge Regression           0.8027         1771.18

The Ridge Regression model achieves the highest accuracy. The strong performance is because Ridge Regression penalizes large coefficients, reducing multicollinearity and overfitting.It maintains most predictive features.